# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('case').getOrCreate()

In [3]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [9]:
index_col = df.select('Cruise_line')

In [12]:
index_col.show(5)

+-----------+
|Cruise_line|
+-----------+
|    Azamara|
|    Azamara|
|   Carnival|
|   Carnival|
|   Carnival|
+-----------+
only showing top 5 rows



In [13]:
from pyspark.ml.feature import StringIndexer
indexed = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_line_indexed')
indexed = indexed.fit(df).transform(df)

In [14]:
indexed.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_indexed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|                1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.5

In [20]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_indexed']

In [21]:
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length',
                                       'cabins','passenger_density','Cruise_line_indexed'], 
                            outputCol='features')

In [45]:
assembler

VectorAssembler_49dfa10ad09af7078295

In [23]:
output = assembler.transform(indexed)

In [24]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_indexed: double (nullable = true)
 |-- features: vector (nullable = true)



In [26]:
final_data = output.select('features','crew')

In [28]:
final_data.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [29]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [30]:
from pyspark.ml.regression import LinearRegression

In [32]:
model = LinearRegression(labelCol='crew')

In [33]:
model = model.fit(train_data)

In [34]:
test_results = model.evaluate(test_data)

In [35]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|   2.117442758860558|
| -0.7882279579820954|
| 0.29990295106471443|
| -1.6009695998079376|
| 0.24573218676215802|
| 0.21716473753750165|
|  -0.714685914958304|
| -0.6722416922263648|
|-0.45679365666946037|
|-0.02625345457924...|
| -0.6291398112326991|
| -0.2568788234773631|
|  2.3237765553946783|
|  0.8036224277436244|
|  0.6553231755250764|
|  0.6962732978545887|
|  -1.410686918989006|
|  0.8938095779868771|
| -0.8891585051245183|
| -0.5294281806243317|
+--------------------+
only showing top 20 rows



In [37]:
test_results.rootMeanSquaredError

0.8697354283097809

In [38]:
test_results.r2

0.9097909816468338

In [40]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [41]:
test_results.meanAbsoluteError

0.6710004457882283

In [43]:
test_results.predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[5.0,122.0,28.5,1...|  6.7| 4.582557241139442|
|[6.0,30.276999999...| 3.55| 4.338227957982095|
|[6.0,93.0,23.94,9...|11.09|10.790097048935285|
|[7.0,89.6,25.5,9....| 9.87|11.470969599807937|
|[8.0,77.499,19.5,...|  9.0| 8.754267813237842|
|[9.0,81.0,21.44,9...| 10.0| 9.782835262462498|
|[9.0,85.0,19.68,9...| 8.69| 9.404685914958304|
|[9.0,105.0,27.2,8...|10.68|11.352241692226364|
|[10.0,58.825,15.6...|  7.0|  7.45679365666946|
|[10.0,68.0,10.8,7...| 6.36| 6.386253454579241|
|[10.0,86.0,21.14,...|  9.2| 9.829139811232698|
|[10.0,90.09,25.01...| 8.58| 8.836878823477363|
|[10.0,151.4,26.2,...|12.53|10.206223444605321|
|[11.0,90.0,22.4,9...| 11.0|10.196377572256376|
|[12.0,77.104,20.0...| 9.59| 8.934676824474924|
|[12.0,88.5,21.24,...|10.29|  9.59372670214541|
|[12.0,88.5,21.24,...|  9.3|10.710686918989007|
|[12.0,91.0,20.32,...| 9.99| 9.096190422

In [46]:
from pyspark.sql.functions import corr

In [49]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

